# Packages

In [8]:
from ipywidgets import HTML
import ipywidgets as widgets
from ipyleaflet import Map, Polyline, Rectangle, basemaps, basemap_to_tiles, Polygon, FullScreenControl, Popup, WidgetControl
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# Functions

In [9]:
def rotate(point, rot):
    
    """Given a 'natural' position of the rectangle printed while representing a ship
    rotate it to the angle it is currently facing
    """
    
    return (point[0]*math.cos(math.radians(rot)) - point[1]*math.sin(math.radians(rot)), 
            point[0]*math.sin(math.radians(rot)) + point[1]*math.cos(math.radians(rot)))

def get_bounds(breadth, length, longitude, latitude, rot_angle):
    """This function gets the real bounds of a boat ==> Realistic plotting
    Knowing that according to: https://en.wikipedia.org/wiki/Decimal_degrees
    Assuming 0.00001deg is equal to 1.1132 m"""
    
    length = length*0.001/1.1132
    breadth = breadth*0.001/1.1132
    
    center_point = np.array((longitude, latitude))
    
    ship_shape_ini = [(-breadth/2, length/2), (breadth/2, length/2), (breadth/2, -length/2), (-breadth/2, -length/2)]
    
    ship_shape_rot = [np.array(rotate(point, rot_angle)) for point in ship_shape_ini]
    
    xy1 = list(ship_shape_rot[0] + center_point)
    xy2 = list(ship_shape_rot[1] + center_point)
    xy3 = list(ship_shape_rot[2] + center_point)
    xy4 = list(ship_shape_rot[3] + center_point)
    
    return [xy1,xy2,xy3,xy4]



def random_hex_color():
    """Get a random color to plot a ship"""
    import random
    r = lambda: random.randint(0,255)
    return ('#%02X%02X%02X' % (r(),r(),r()))

def get_color_scale(arr, scale_type='plasma'):
    """
    Function to assign a scale color within plasma predefined scale to plot a feature using color
    """
    arr = list(arr)
    #https://matplotlib.org/gallery/color/colormap_reference.html
    scale = cm.get_cmap(scale_type, len(arr)).colors
    
    arr = np.array(arr)
    sorted_index = np.argsort(arr)
    return {arr[sorted_index[i]]:rgb_to_hex(scale[i]) for i in range(len(arr))}

def rgb_to_hex(rgb):
    if len(rgb) == 4:
        rgb = rgb[:3]
    return '#%02x%02x%02x' % tuple(list([int(elem*256) for elem in rgb]))

# Loading DataSet

In [10]:
filename = '/datc/saab/reduced_area.h5'
data = pd.read_hdf(filename, 'df')

## Visualize the 5 first rows of data

In [11]:
data = data.dropna()
data.head()

data.latitude = data['latitude'] + 47.72
data.longitude = data['longitude'] + 157.85
data

,mmsi,datetime,latitude,longitude,orientation,rateofturn,course,length,breadth,speed,vesseltype
171,56295,2018-11-30 16:00:00.707,22.199854,114.080508,98.0000,0.000000,89.000000,24.50000,3.099609,0.620117,0
172,56946,2018-11-30 16:00:00.707,22.170015,114.079086,322.5000,0.000000,6.398438,26.00000,7.000000,5.078125,0
188,54494,2018-11-30 16:00:00.707,22.267552,114.092355,56.3125,0.000000,53.000000,63.00000,12.000000,7.519531,0
189,0,2018-11-30 16:00:00.707,22.216311,114.091476,166.2500,0.000000,165.000000,54.81250,13.703125,3.759766,0
190,0,2018-11-30 16:00:00.707,22.191633,114.090895,275.2500,0.000000,136.375000,148.75000,19.203125,0.040009,0
...,...,...,...,...,...,...,...,...,...,...,...
42501852,11655,2018-12-01 15:41:15.706,22.247677,114.148988,0.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0
42501854,54367,2018-12-01 15:41:15.706,22.247290,114.148953,16.0000,0.000000,0.000000,0.00000,0.000000,0.000000,0
42501855,11797,2018-12-01 15:41:15.706,22.244843,114.125698,122.8125,3.400391,81.500000,24.59375,3.400391,4.589844,0
42501861,54044,2018-12-01 15:41:15.706,22.250201,114.131207,145.3750,0.000000,146.375000,14.00000,4.000000,8.601562,0


# Data visualization (map)

### Define variables of the visualization

In [12]:
CRAFT_length_list = data.length.unique()
color_scale = get_color_scale(CRAFT_length_list, )

CRAFT_ID_list = data.mmsi.unique()#Get the mmsi unique values into a list:
CRAFT_ID_list = CRAFT_ID_list[CRAFT_ID_list!=0]

In [13]:
ships_info = [[[] for  i in range(31)] for j in range(len(CRAFT_ID_list))]#List that will storage a list of lists == a list of time-series(which will as well be represented as a list)

ship_number = 0
for rowid in CRAFT_ID_list:
    #Start with empty lists
    npinfo, infolist = [], []
    #Get a numpy array composed by 'latitude', 'longitude', 'orientation', 'length', 'breadth'
    npinfo = data[data.mmsi == rowid][['latitude', 'longitude', 'orientation', 'length', 'breadth', 'datetime']].values
    #Convert it to a python list so it can be an attribute of the multypoligon functionality of ipyleaflet
    infolist = coordslist = list([list(coords) for coords in npinfo])
    for point in infolist:
        ships_info[ship_number][int(str(point[5]).split('-')[2][:2]) - 1].append(point)
    ship_number+=1

### Visualize

In [20]:
m = Map(center = (22.205232, 114.123882), zoom = 12)#Define the map object

#To define  the maximum number of steps we will be able to take with the slider
max_steps = max([len(element) for element in ships_info])
ships_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(ships_info),
    step=1,
    description='Ships: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
widget_steps = WidgetControl(widget=ships_slider, position='topright')
m.add_control(widget_steps)
m.add_control(FullScreenControl())
#dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
#m.add_layer(dark_matter_layer)
previous_value = 0


def update_map(ships_slider):
    global previous_value, m
    if previous_value > ships_slider:
        m = Map(center = (-22.884059, 133.714373), zoom =4)#Define the map object
        ini, end = 0, ships_slider
    else:
        ini, end = previous_value, ships_slider
        
        
    for i in range(ini, end):
        color_value = '#000000'
        #for each time series in ships_info list --> Paint The line
        points = [ships_info[i][29][j] for j in range(len(ships_info[i][29])) if len(ships_info[i][29]) < 5000]
        line = Polyline(
            locations = [[elem[0],elem[1]] for elem in ships_info[i][29]],
            color = color_value,
            fill_color= "transparent",
            weight = 3,
            opacity = 0.1)

        m.add_layer(line)
        """        if ships_info[i][29] != []:
            singular_info = ships_info[i][29][-1]
            message = HTML()
            message.value = '<b>**DATE: <br>' + str(singular_info[-1]) + '**</b><br><b>Longitude</b>: ' + str(singular_info[0]) + '<br><b>Latitude</b>:  ' + str(singular_info[1]) + "<br><b>Vessel's Length</b>: " + str(singular_info[3]) + "<br><b>Vessel's Breadth</b>: " + str(singular_info[4]) + "<br><b>Facing:</b>: " + str(singular_info[2]) + ' deg'

            mapbounds = get_bounds(singular_info[3], singular_info[4], singular_info[0], singular_info[1], singular_info[2])
            ship_shape = Polygon(
            locations=[mapbounds],
            color=color_value,
            fill_color=color_value,
            fill_opacity=1
            )
            ship_shape.popup = message
            m.add_layer(ship_shape)"""
    previous_value = ships_slider
display(m)
widgets.interactive(update_map, ships_slider=ships_slider)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

interactive(children=(IntSlider(value=0, continuous_update=False, description='Ships: ', max=581), Output()), …

In [26]:
from ipywidgets.embed import embed_minimal_html, dependency_state
embed_minimal_html('density_plot_reduced_area_dynamic' + '.html', views=[m])

# Reusable Function To Visualize Ships by IDs

In [33]:
def visualize_by_id(ID_list, data, normal = [], medium = [], anomalous = []):
    #List that will storage a list of lists == a list of time-series(which will as well be represented as a list)
    ships_info = [[[] for  i in range(31)] for j in range(len(ID_list))]
    
    ship_number = 0
    #For each ship in ID_list
    for rowid in ID_list:
        #Start with empty lists
        npinfo, infolist = [], []
        #Get a numpy array composed by 'latitude', 'longitude', 'orientation', 'length', 'breadth'
        npinfo = data[data.mmsi == rowid][['latitude', 'longitude', 'orientation', 'length', 'breadth', 'datetime']].values
        #Convert it to a python list so it can be an attribute of the multypoligon functionality of ipyleaflet
        infolist = coordslist = list([list(coords) for coords in npinfo])
        for point in infolist:
            ships_info[ship_number][int(str(point[5]).split('-')[2][:2]) - 1].append(point)
        ship_number+=1
        
    #Create the map
    m = Map(center = (22.205232, 114.123882), zoom = 12)#Define the map object
    
    #Define the slider widget
    ships_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(ships_info),
        step=1,
        description='Ships: ',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
    
    #Add the slider to the map
    widget_steps = WidgetControl(widget=ships_slider, position='topright')
    m.add_control(widget_steps)
    m.add_control(FullScreenControl())
    #dark_matter_layer = basemap_to_tiles(basemaps.CartoDB.DarkMatter)
    #m.add_layer(dark_matter_layer)
    previous_value = 0
    

    for i in range(0, len(ID_list)):
        color = '#000000'
        if ID_list[i] in normal:
            color_value = '#4FEA5D'
        elif ID_list[i] in medium:
            color_value = '#E8CE54'
        elif ID_list[i] in anomalous:
            color_value = '#F74343'
            
        #for each time series in ships_info list --> Paint The line
        points = [ships_info[i][29][j] for j in range(len(ships_info[i][29])) if len(ships_info[i][29]) < 5000]
        
        line = Polyline(
            locations = [[elem[0],elem[1]] for elem in ships_info[i][29]],
            color = color_value,
            fill_color= "transparent",
            weight = 3,
            opacity = 1)

        m.add_layer(line)
        """        if ships_info[i][29] != []:
            singular_info = ships_info[i][29][-1]
            message = HTML()
            message.value = '<b>**DATE: <br>' + str(singular_info[-1]) + '**</b><br><b>Longitude</b>: ' + str(singular_info[0]) + '<br><b>Latitude</b>:  ' + str(singular_info[1]) + "<br><b>Vessel's Length</b>: " + str(singular_info[3]) + "<br><b>Vessel's Breadth</b>: " + str(singular_info[4]) + "<br><b>Facing:</b>: " + str(singular_info[2]) + ' deg'

            mapbounds = get_bounds(singular_info[3], singular_info[4], singular_info[0], singular_info[1], singular_info[2])
            ship_shape = Polygon(
            locations=[mapbounds],
            color=color_value,
            fill_color=color_value,
            fill_opacity=1
            )
            ship_shape.popup = message
            m.add_layer(ship_shape)"""
    previous_value = ships_slider

    return m


# Test the function

In [34]:
m = visualize_by_id(CRAFT_ID_list[:200] , data, normal=CRAFT_ID_list[:150], medium = CRAFT_ID_list[150:180], anomalous=CRAFT_ID_list[180:200])

Visualize the results

In [35]:
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

### Save the map

In [36]:
from ipywidgets.embed import embed_minimal_html, dependency_state
embed_minimal_html('reusable_function_with_colors' + '.html', views=[m])